# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,21.72,95,0,1.03,ZA,1707079878
1,1,adamstown,-25.0660,-130.1015,24.99,79,98,4.50,PN,1707079878
2,2,sabha,27.0377,14.4283,12.95,44,75,3.49,LY,1707079878
3,3,melekeok village,7.4957,134.6367,27.31,83,75,3.60,PW,1707079878
4,4,margaret river,-33.9500,115.0667,13.05,78,59,2.93,AU,1707079879


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_city_humidity = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size='Humidity',
    frame_width=800,
    frame_height=600,
    hover_cols=['City'],
    color = 'City'
)

# Display the map
map_plot_city_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] >= 21) & # ~ 70F 
                               (city_data_df['Max Temp'] <= 32) & # ~ 90F
                               (city_data_df['Cloudiness'] <= 50) & # 50% is the most
                               (city_data_df['Lat'] < 0) & # Southern Hemisphere
                               (city_data_df['Humidity'] <= 60)] # 60% humidity is the most

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,oamaru,-45.0784,170.9801,21.17,52,48,3.70,NZ,1707079896
69,69,alice springs,-23.7000,133.8833,28.75,33,37,4.12,AU,1707079898
148,148,chonchi,-42.6128,-73.8092,25.56,34,1,1.36,CL,1707079908
152,152,senillosa,-39.0000,-68.4167,31.88,21,0,11.76,AR,1707079909
179,179,carnarvon,-24.8667,113.6333,22.80,57,1,10.24,AU,1707079726
281,281,palpa,-14.5336,-75.1856,29.86,53,42,5.64,PE,1707079929
297,297,calvinia,-31.4707,19.7760,22.03,41,3,2.21,ZA,1707079931
304,304,ingeniero jacobacci,-41.3427,-69.5448,29.89,16,47,11.28,AR,1707079756
341,341,valdivia,-39.8142,-73.2459,28.05,32,0,3.66,CL,1707079936
348,348,upington,-28.4478,21.2561,30.21,30,38,5.14,ZA,1707079937


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
56,oamaru,NZ,-45.0784,170.9801,52,
69,alice springs,AU,-23.7000,133.8833,33,
148,chonchi,CL,-42.6128,-73.8092,34,
152,senillosa,AR,-39.0000,-68.4167,21,
179,carnarvon,AU,-24.8667,113.6333,57,
281,palpa,PE,-14.5336,-75.1856,53,
297,calvinia,ZA,-31.4707,19.7760,41,
304,ingeniero jacobacci,AR,-41.3427,-69.5448,16,
341,valdivia,CL,-39.8142,-73.2459,32,
348,upington,ZA,-28.4478,21.2561,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
oamaru - nearest hotel: Brydone Hotel Oamaru
alice springs - nearest hotel: Aurora Alice Springs
chonchi - nearest hotel: Hotel Huildín
senillosa - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
palpa - nearest hotel: Alubias
calvinia - nearest hotel: No hotel found
ingeniero jacobacci - nearest hotel: Gran Hotel Argentino
valdivia - nearest hotel: Esmeralda
upington - nearest hotel: Die Eiland Holiday Ressort
villa de soto - nearest hotel: La Eleida
calama - nearest hotel: Apart Hotel Geotel Calama
ambon - nearest hotel: Hotel Hero
despenaderos - nearest hotel: No hotel found
las animas - nearest hotel: Hotel Villa del Río


,City,Country,Lat,Lng,Humidity,Hotel Name
56,oamaru,NZ,-45.0784,170.9801,52,Brydone Hotel Oamaru
69,alice springs,AU,-23.7000,133.8833,33,Aurora Alice Springs
148,chonchi,CL,-42.6128,-73.8092,34,Hotel Huildín
152,senillosa,AR,-39.0000,-68.4167,21,No hotel found
179,carnarvon,AU,-24.8667,113.6333,57,No hotel found
281,palpa,PE,-14.5336,-75.1856,53,Alubias
297,calvinia,ZA,-31.4707,19.7760,41,No hotel found
304,ingeniero jacobacci,AR,-41.3427,-69.5448,16,Gran Hotel Argentino
341,valdivia,CL,-39.8142,-73.2459,32,Esmeralda
348,upington,ZA,-28.4478,21.2561,30,Die Eiland Holiday Ressort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Country', 'Hotel Name'],
    color = 'City'
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)